Import all libraries

In [35]:
import requests
import json
import time
import multiprocessing
import psycopg2


Functions for loading the data and counting rows in meanwhile

In [1]:
# Postgres
def load_data_postgres(data):
    start = time.time()
    for line in data:
        response_insert = requests.post('http://localhost:3000/postgres/write/crypto',json=line).text
    end = time.time()
    print("postgres",end-start)

# Firebase
def load_data_firebase(data):
    start = time.time()
    prev_time = start
    for line in data:
        response_insert = requests.post('http://localhost:3000/firebase/write/crypto',json=line).text
    end = time.time()
    print("firebase",end-start)

def time_measurement(num,l):
    start = time.time()
    prev_time = start
    for i in range(num//10):
        dt = time.time() - prev_time
        if dt > 20:
            # print or write to log here
            prev_time = time.time()   
            response_write = int(requests.get('http://localhost:3000/postgres/read/crypto').text)
            print(response_write)
            #running queries after 50 sec
            query_postgres(l)




In [2]:
def query_postgres(query_list):
    q_all_start = time.time()
        
    for i,que in enumerate(query_list):
        temp = {}
        temp['query'] = que
        start = time.time()
        print(temp)
        
        # post request <<
        response_query = requests.post('http://localhost:3000/postgres/query/crypto',json=temp)
        print(response_query)
        
        end = time.time()
        print("time taken for query {} is {}".format(i+1,(end-start)))
        
    q_all_end = time.time()
    print("time taken to run all the queries", q_all_end-q_all_start)

def query_firebase(query_list):
    q_all_start = time.time()
        
    for i,que in enumerate(query_list):
        temp = {}
        temp['query'] = que
        start = time.time()
        print(temp)
        
        # post request <<
        response_query = requests.post('http://localhost:3000/firebase/query/crypto',json=temp)
        print(response_query)
        
        end = time.time()
        print("time taken for query {} is {}".format(i+1,(end-start)))
        
    q_all_end = time.time()
    print("time taken to run all the queries", q_all_end-q_all_start)

In [3]:
# l = ["""select count(*) from crypto_tab;""",
#      """ select * from crypto_tab order by bitcoin_info->>'Date' DESC limit 100;""",
#     """select count(*) from crypto_tab group by bitcoin_info->>'Symbol';""",
#      """select * from crypto_tab where (bitcoin_info->>'Volume')::float > 2;"""
#     """select * from crypto_tab where bitcoin_info->>'Date' = '2018-12-07 21:52:00';""",
#     """select * from crypto_tab where (bitcoin_info->>'High')::float > 3800 and  bitcoin_info->>'Symbol' = 'BTCUSD';"""
#     ]
# query_postgres(l)

In [4]:
l = ["""select count(*) from crypto_tab;""",
     """ select * from crypto_tab order by bitcoin_info->>'Date' DESC limit 100;""",
    """select * from crypto_tab where bitcoin_info->>'Date' = '2020-12-31 21:52:00';""",
    """select * from crypto_tab where (bitcoin_info->>'High')::float > 3800 and (bitcoin_info->>'High')::float < 4500; """,
     """select * from crypto_tab where (bitcoin_info->>'Volume')::float > 2;"""
    ]
# query_postgres(l)

Select which data to load

In [5]:
# BITCOIN DATA
json_file_path= './data/bitcoin_data_sample.json'
with open(json_file_path) as json_file:
    bitcoin_data = json.load(json_file)

In [ ]:
# ETHEREUM DATA
json_file_path= './data/bitcoin_data_sample.json'
with open(json_file_path) as json_file:
    ethereum_data = json.load(json_file)

In [56]:
# LITECOIN DATA
json_file_path= './bitcoin_merged_data.json'
with open(json_file_path) as json_file:
    data = json.load(json_file)

In [59]:
load_data_postgres(data)

In [ ]:
time_measurement(l)

Run process for Firebase

In [8]:

p1 = multiprocessing.Process(target=load_data_postgres, args=(bitcoin_data, ))
p2 = multiprocessing.Process(target=load_data_postgres, args=(bitcoin_data, ))
p3 = multiprocessing.Process(target=load_data_postgres, args=(bitcoin_data, ))
p4 = multiprocessing.Process(target=time_measurement, args=(len(bitcoin_data), ))

# starting process 1
p1.start()
# starting process 2
p2.start()
# starting process 3
p3.start()
# starting process 4
p4.start()

# wait until process 1 is finished
p1.join()
# wait until process 2 is finished
p2.join()
# wait until process 3 is finished
p3.join()
# wait until process 4 is finished
p4.join()

Run process for PostgreSQL

In [10]:
load_data_postgres(data)

postgres 0.7907586097717285


In [38]:
# BITCOIN DATA
json_file_path= './data/trajectory_data_60k.json'
with open(json_file_path) as json_file:
    data = json.load(json_file)

In [43]:
len(data)

60000

In [47]:
sql = """insert into trajectory_path(path_info) values(%s)"""


 

sql_count = """select count(*) from trajectory_path"""


 


start_time = time.time()
prev_time = start_time
prv_count = 0


with psycopg2.connect(host='localhost', port=5432, database='applicationdb', user='postgres', password='4258') as conn:

    with conn.cursor() as cur:


        

        # q_all_start = time.time()

        # for i,que in enumerate(l):
        # temp = {}
        # temp['query'] = que
        # start = time.time()

        # # post request <<
        # cur.execute(que)
        # # post request >>

        # end = time.time()
        # print("time taken for query {} is {}".format(i+1,(end-start)))

        # q_all_end = time.time()
        # print("time taken to run all the queries", q_all_end-q_all_start)


        #start_time = time.time()
        print(start_time)
        counter = 0
        for line in data:
            line = str(line)
            line = line.replace('\'','\"')
            try:
                x = cur.execute(sql, (line,))
            except (Exception, psycopg2.DatabaseError) as error:
                print(error)
                print(time.time()-start_time)
                continue


    

            dt = time.time() - prev_time
            if dt > 1:
            # print or write to log here
                prev_time = time.time()
                cur.execute(sql_count)
                result = cur.fetchone()
                print(result[0])


        conn.commit()

    # except (Exception, psycopg2.DatabaseError) as error:
    # print(error)
    # print(time.time()-start_time)
    end_time = time.time()


    

    print(end_time)
    print("time taken to load the data is {} seconds".format(end_time-start_time))

1637452712.3010046
2058
4605
7130
9652
12216
14725
17256
19806
22350
24918
27493
30004
32538
35064
37589
40129
42657
45291
47736
50190
52728
missile
CONTEXT:  PL/pgSQL function process_dist_calculation() line 4 at RAISE

21.097785234451294
current transaction is aborted, commands ignored until end of transaction block

21.098284006118774
current transaction is aborted, commands ignored until end of transaction block

21.09968662261963
current transaction is aborted, commands ignored until end of transaction block

21.100196361541748
current transaction is aborted, commands ignored until end of transaction block

21.101358890533447
current transaction is aborted, commands ignored until end of transaction block

21.102198123931885
current transaction is aborted, commands ignored until end of transaction block

21.102198123931885
current transaction is aborted, commands ignored until end of transaction block

21.103246450424194
current transaction is aborted, commands ignored until end of